# Kernel time series prediction

This notebook generates reference results on all tree data sets using _Time Series Prediction for Graphs in Kernel and Dissimilarity Spaces_ ([Paaßen, Göpfert, and Hammer, 2018](https://arxiv.org/abs/1704.06498)). This method performs a prediction in kernel space as follows. Let $\{(x_i, y_i)\}_{i \in \{1, \ldots, m\}}$ be the training data where $y_i$ is the true successor of $x_i$. Then, the prediction $\phi(y)$ in kernel space for the input tree $x$ is given as

$$\phi(y) = \phi(x) + \sum_i \gamma_i \cdot [ \phi(y_i) - \phi(x_i) ]$$

where $\vec \gamma = \big( K + \sigma^2 \cdot I \big)^{-1} \cdot \vec k(x)$ are the coefficients computed by Gaussian process/kernel prediction, where $K$ is the kernel matrix between all training data and where $\vec k(x)$ is the vector of kernel values from $x$ to all training trees. The kernel we use here is $k(x, y) = \exp\big(-\frac{1}{2 \psi^2} \cdot d(x, y)^2 \big)$, where $d$ is the tree edit distance and $\psi$ is a hyper-parameter called bandwidth. This is the suggested kernel of ([Paaßen, Göpfert, and Hammer, 2018](https://arxiv.org/abs/1704.06498)).

To map back from kernel space to a tree we employ the scheme recommended by the authors in ([Paaßen et al., 2018](https://jedm.educationaldatamining.org/index.php/JEDM/article/view/158)). In particular, we greedily apply edits $\delta$ to $x$ that move towards training data trees $y_i$ with positive coefficients $\gamma_i$ or towards training data trees $x_i$ with negative coefficients $\gamma_i$ and which reduce the loss

$$\ell(\delta) = d(x, \delta(x))^2 + \sum_i \gamma_i [ d(\delta(x), y_i)^2 - d(\delta(x), x_i)^2) ]$$

until no edit $\delta$ is found anymore which reduces the loss.

For even more details, please refer to the source code at `kernel_time_series_prediction.py`.

## Boolean Formulae and Peano Addition

Refer to `boolean_formulae.py` and `peano_additon.py` for details on the datasets.

In [1]:
# evaluate kernel tree time series prediction on both tasks in five repeats
import json
import os
import random
import time
import numpy as np
import python_ast_utils
import edist.tree_utils as tu
import edist.tree_edits as te
import edist.ted as ted
import boolean_formulae
import peano_addition
from kernel_time_series_prediction import KernelTreePredictor

R = 5
# the number of time series we generate for training
M = 100
# the number of time series for testing
N_test = 10

# model hyper parameters
psi   = None
sigma = 1E-3

tasks = ['boolean', 'peano']

errors = np.zeros((len(tasks), R))
baseline_errors = np.zeros((len(tasks), R))
times  = np.zeros((len(tasks), R))
prediction_times  = np.zeros((len(tasks), R))

# iterate over all tasks
for task_idx in range(len(tasks)):
    task = tasks[task_idx]
    print('\n\n --- task %s --- \n' % task)
    if task == 'boolean':
        sampling_fun = boolean_formulae.generate_time_series
    else:
        sampling_fun = peano_addition.generate_time_series

    # do repeats
    for r in range(R):
        print('repeat %d of %d' % (r+1, R))
        # sample random training data
        training_data = []
        for i in range(M):
            training_data.append(sampling_fun())
        # fit model
        start = time.time()
        model = KernelTreePredictor(psi = psi, sigma = sigma)
        model.fit(training_data)
        times[task_idx, r] = time.time() - start
        print('took %g seconds to train' % times[task_idx, r])
        # evaluate it on the test data
        rmse = 0.
        baseline_rmse = 0.
        m = 0
        for i in range(N_test):
            # sample test time series
            time_series = sampling_fun()
            for t in range(len(time_series)-1):
                nodes, adj = time_series[t][0], time_series[t][1]
                # predict next tree
                start = time.time()
                _, nodes_pred, adj_pred = model.predict(nodes, adj)
                prediction_times[task_idx, r] += time.time() - start
                # compare to actual next tree
                nodes_act, adj_act = time_series[t+1][0], time_series[t+1][1]
                d = ted.ted(nodes, adj, nodes_act, adj_act)
                baseline_rmse += d * d
                if nodes_pred != nodes_act or adj_pred != adj_act:
                    d = ted.ted(nodes_pred, adj_pred, nodes_act, adj_act)
                    rmse += d * d
            m += len(time_series)
        rmse = np.sqrt(rmse / m)
        baseline_rmse = np.sqrt(baseline_rmse / m)
        prediction_times[task_idx, r] /= m
        print('took %g seconds to predict' % prediction_times[task_idx, r])
        errors[task_idx, r] = rmse
        baseline_errors[task_idx, r] = baseline_rmse
        print('RMSE: %g versus baseline RMSE %g' % (rmse, baseline_rmse))



 --- task boolean --- 

repeat 1 of 5
took 0.137753 seconds to train
took 0.470501 seconds to predict
RMSE: 2.41091 versus baseline RMSE 1.45774
repeat 2 of 5
took 0.129635 seconds to train
took 0.469569 seconds to predict
RMSE: 2.43812 versus baseline RMSE 1.61589
repeat 3 of 5
took 0.135248 seconds to train
took 0.419243 seconds to predict
RMSE: 2.46306 versus baseline RMSE 1.52753
repeat 4 of 5
took 0.130834 seconds to train
took 0.215008 seconds to predict
RMSE: 2 versus baseline RMSE 1.7097
repeat 5 of 5
took 0.131168 seconds to train
took 0.382734 seconds to predict
RMSE: 2.65684 versus baseline RMSE 1.59041


 --- task peano --- 

repeat 1 of 5
took 3.70749 seconds to train
took 58.3135 seconds to predict
RMSE: 5.38069 versus baseline RMSE 3
repeat 2 of 5
took 4.74174 seconds to train
took 65.4441 seconds to predict
RMSE: 3.96863 versus baseline RMSE 2.79322
repeat 3 of 5
took 3.83915 seconds to train
took 61.2994 seconds to predict
RMSE: 4.89415 versus baseline RMSE 3.44944
r

In [2]:
# store the experimental results
np.savetxt('results/kernel_synthetic_times.csv', times.T, delimiter='\t', header='\t'.join(tasks), fmt='%g', comments = '')
np.savetxt('results/kernel_synthetic_predict_times.csv', prediction_times.T, delimiter='\t', header='\t'.join(tasks), fmt='%g', comments = '')
np.savetxt('results/kernel_synthetic_rmses.csv', errors.T, delimiter='\t', header=('\t'.join(tasks)), fmt='%g', comments = '')
np.savetxt('results/kernel_synthetic_baseline_rmses.csv', baseline_errors.T, delimiter='\t', header=('\t'.join(tasks)), fmt='%g', comments = '')

## Python Datasets

In [1]:
# evaluate kernel tree time series prediction on all tasks in crossvalidation
import json
import os
import random
import time
import numpy as np
import python_ast_utils
import edist.tree_utils as tu
import edist.tree_edits as te
import edist.ted as ted
from kernel_time_series_prediction import KernelTreePredictor

psi   = None
sigma = 1E-3
num_folds     = 5

tasks = python_ast_utils._tasks + ['pysort']

crossval_errors = np.zeros((len(tasks), num_folds))
baseline_errors = np.zeros((len(tasks), num_folds))
crossval_times  = np.zeros((len(tasks), num_folds))
crossval_prediction_times  = np.zeros((len(tasks), num_folds))

# iterate over all tasks
for task_idx in range(len(tasks)):
    task = tasks[task_idx]
    print('\n\n --- task %s --- \n' % task)
    if task == 'pysort':
        data = python_ast_utils.load_pysort()
    else:
        data = python_ast_utils.load_task_cleaned(task)

    # load the folds
    with open('results/%s_folds.json' % task) as folds_file:
        folds = json.load(folds_file)

    if len(folds) != num_folds:
        raise ValueError('Expected %d folds but got %d from file %s' % (num_folds, len(folds), 'results/%s_folds.json' % task))

    # iterate over all folds
    for f in range(num_folds):
        train_data_idxs = folds[f][0]
        test_data_idxs  = folds[f][1]
        print('\n---fold %d of %d ---' % (f+1, num_folds))
        print('train_idxs: %s' % str(train_data_idxs))
        print('test_idxs: %s' % str(test_data_idxs))
        # get training and test data for the current fold
        train_data = []
        for i in train_data_idxs:
            train_data.append(data[i])
        test_data = []
        for i in test_data_idxs:
            test_data.append(data[i])
        # train the kernel regression model
        start = time.time()
        model = KernelTreePredictor(psi = psi, sigma = sigma)
        model.fit(train_data)
        crossval_times[task_idx, f] = time.time() - start
        print('took %g seconds to train' % crossval_times[task_idx, f])
        # evaluate it on the test data
        rmse = 0.
        baseline_rmse = 0.
        m = 0
        for i in range(len(test_data)):
            time_series = test_data[i]
            for t in range(len(time_series)):
                nodes, adj = time_series[t][0], time_series[t][1]
                # predict next tree
                start = time.time()
                _, nodes_pred, adj_pred = model.predict(nodes, adj)
                crossval_prediction_times[task_idx, f] += time.time() - start
                # compare to actual next tree
                if t < len(time_series)-1:
                    nodes_act, adj_act = time_series[t+1][0], time_series[t+1][1]
                else:
                    nodes_act, adj_act = nodes, adj
                d = ted.ted(nodes, adj, nodes_act, adj_act)
                baseline_rmse += d * d
                if nodes_pred != nodes_act or adj_pred != adj_act:
                    d = ted.ted(nodes_pred, adj_pred, nodes_act, adj_act)
                    rmse += d * d
            m += len(time_series)
        rmse = np.sqrt(rmse / m)
        baseline_rmse = np.sqrt(baseline_rmse / m)
        crossval_prediction_times[task_idx, f] /= m
        print('took %g seconds to predict' % crossval_prediction_times[task_idx, f])
        crossval_errors[task_idx, f] = rmse
        baseline_errors[task_idx, f] = baseline_rmse
        print('RMSE: %g versus baseline RMSE %g' % (rmse, baseline_rmse))



 --- task fun --- 


---fold 1 of 5 ---
train_idxs: [0, 1, 2, 3, 4, 5, 7, 8, 10, 11, 13, 14]
test_idxs: [6, 9, 12]
took 0.1365 seconds to train
took 0.338039 seconds to predict
RMSE: 10.7319 versus baseline RMSE 6.245

---fold 2 of 5 ---
train_idxs: [0, 1, 3, 4, 6, 8, 9, 10, 11, 12, 13, 14]
test_idxs: [2, 5, 7]
took 0.141758 seconds to train
took 0.437477 seconds to predict
RMSE: 8.08775 versus baseline RMSE 5.97052

---fold 3 of 5 ---
train_idxs: [0, 1, 2, 4, 5, 6, 7, 9, 10, 11, 12, 13]
test_idxs: [3, 8, 14]
took 0.141499 seconds to train
took 0.53131 seconds to predict
RMSE: 11.7724 versus baseline RMSE 7.74597

---fold 4 of 5 ---
train_idxs: [0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 12, 14]
test_idxs: [4, 11, 13]
took 0.148223 seconds to train
took 0.421412 seconds to predict
RMSE: 9.56794 versus baseline RMSE 7.55585

---fold 5 of 5 ---
train_idxs: [2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14]
test_idxs: [0, 1, 10]
took 0.145582 seconds to train
took 0.452888 seconds to predict
RMSE: 8.28471 ve

KeyboardInterrupt: 

In [8]:
# store the experimental results

np.savetxt('results/kernel_crossval_times.csv', crossval_times.T, delimiter='\t', header='\t'.join(tasks), fmt='%g', comments = '')
np.savetxt('results/kernel_crossval_predict_times.csv', crossval_prediction_times.T, delimiter='\t', header='\t'.join(tasks), fmt='%g', comments = '')
np.savetxt('results/kernel_crossval_rmses.csv', crossval_errors.T, delimiter='\t', header=('\t'.join(tasks)), fmt='%g', comments = '')
np.savetxt('results/kernel_crossval_baseline_rmses.csv', baseline_errors.T, delimiter='\t', header=('\t'.join(tasks)), fmt='%g', comments = '')